In [1]:
print("ok")

ok


In [3]:


%pwd

'c:\\Users\\HP User\\Desktop\\Medical_chatbot\\Medical_Chatbot-GenAi\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\HP User\\Desktop\\Medical_chatbot\\Medical_Chatbot-GenAi'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# extracting Data from the Pdf
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob = "*.pdf", loader_cls = PyPDFLoader)
    
    documents  = loader.load()
    return documents


In [8]:
extracted_data = load_pdf_file(data = 'Data/')

In [9]:
# extracted_data

In [10]:
# split the data into chunks

def text_split(extracted_data):
     text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20)
     text_chunks = text_splitter.split_documents(extracted_data)
     return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("lenght of chunks", len(text_chunks))

lenght of chunks 6973


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings


In [13]:
# Downloading Embedding model from hugging face
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2")


C:\Users\HP User\AppData\Local\Temp\ipykernel_6408\3078201810.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2")
c:\Users\HP User\anaconda3\envs\stenvt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("lenght: ", len(query_result))

lenght:  384


In [15]:
query_result

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [16]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC


In [21]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC

import os
pc = Pinecone(api_key="pcsk_5NvqbM_MZvRvXhWBMW8C9TfS1c8QC3mFpcSWgpBEZxnqHdoXqhqtC81sgeRjijkDf5oHYg")

index_name = "testbot"

pc.create_index(
    name = index_name,
    dimension = 384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)


{
    "name": "testbot",
    "metric": "cosine",
    "host": "testbot-hffo38h.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [22]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [23]:
PINECONE_API_KEY= os.getenv('PINECONE_API_KEY')
TOGETHER_API_KEY= os.getenv('TOGETHER_API_KEY')

In [24]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY 

In [26]:

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings,
    
    
)

In [117]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings,
)

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retriever_docs = retriever.invoke("what is Acne?")

In [29]:
retriever_docs


[Document(id='2809bb3d-82c8-470f-a5da-c31e7120361d', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='25f15905-0d05-4832-a001-d504291940dc', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator

In [30]:
from langchain_community.llms import Together
llm = Together(
    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    temperature=0.3,
    max_tokens=600,
)

C:\Users\HP User\AppData\Local\Temp\ipykernel_6408\1468887455.py:2: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-aswering task. "
    "Use the following peices of retrieved context to answer"
    "the question. If you dont know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer brief and add definations , info columns, bullet"
    "points and explain different aspects of questions by yourself add as much detail as needed."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [32]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?!"})
print(response["answer"])

?!
Okay, so I need to figure out what Acromegaly and Gigantism are. I remember hearing about them in biology class, but I'm a bit fuzzy on the details. Let me start by recalling what I know. Both terms sound like they have to do with growth, maybe excessive growth? I think they might be related to hormones because a lot of growth issues are hormonal.

Looking at the context provided, I see mentions of the pituitary gland, which is a pea-sized gland at the base of the brain. It produces hormones that affect growth and body functions. So, if something goes wrong with the pituitary gland, it could cause growth issues. I also see terms like glucocorticoids and mineralocorticoids, which are types of hormones, so maybe Acromegaly and Gigantism are related to overproduction of certain hormones.

I remember that Acromegaly is when the body produces too much growth hormone after normal growth has stopped, leading to enlarged features. But wait, isn't that more in adults? And Gigantism is simila